In [ ]:
import pandas as pd
from datetime import datetime
#Loading dictionary and dataset

source=r"/content/drive/MyDrive/Observatorio Fiscal Conjunto/Gasto Público/SAPO/Otros/datos_paraguay/datos/"
dictionary=pd.read_excel(source+r"Diccionario de datos/Diccionario.xlsx")
setsize=6400
data=pd.read_excel(source+r"resultshugging.xlsx",nrows=setsize)

In [ ]:
#dictionary preparation
dictionary=pd.read_excel(source+r"Diccionario de datos/Diccionario.xlsx")
dictionary=dictionary[dictionary["keep"]==1]
dictionary=dictionary.set_index("Original name")
dicc=dictionary["Column"].to_dict()


In [ ]:
def regla(x,sup,inf):
  #this function gives a kind of alert
  if x> sup:
    return "Overpriced"
  elif x>inf:
    return "According"
  else:
    return "Underprice"


In [ ]:
##We make some postprocessing solutions and build on knowledge range

data=data.rename(columns=dicc)[dicc.values()]
data["ID-Contrato"]=data["Tender ID"].apply(lambda x:x.split("-")[-2])
data["Precio Predecido Normalizado Millones"]=data["Precio Predecido"]/1e+12
data["Tender Price millions"]=data["Tender Price"]/1e+6
data["Tender Price millions"]=data["Tender Price millions"].round()

standard_deviation=data["Precio Predecido Normalizado Millones"].std()

data["Franja Superior"]=data["Precio Predecido Normalizado Millones"].apply(
    lambda x:x+0.1*standard_deviation)
data["Franja inferior"]=data["Precio Predecido Normalizado Millones"].apply(
    lambda x:x-0.1*standard_deviation)

data["Reference Range"]=data.apply(lambda row: "["+str(round(row["Franja inferior"]))+","+str(round(row["Franja Superior"]))+"]",axis=1)

data["Alerta"]=data.apply(
    lambda row: regla(
    row["Tender Price millions"],row["Franja Superior"],row["Franja inferior"]),axis=1)



In [ ]:
data=data.dropna(subset="Application Dead line")

In [ ]:
#we simplify the date structure
data["Application Dead Line day"]=data["Application Dead line"].apply(
    lambda x:datetime.strftime(datetime.strptime(str(x).split("T")[0],"%Y-%m-%d"),"%d-%m-%y")
)

In [ ]:
data1=data[["ID-Contrato","Description","Application Dead Line day","Tender Price millions","Reference Range","Caller","Alerta"]]
data1

,ID-Contrato,Description,Application Dead Line day,Tender Price millions,Reference Range,Caller,Alerta
0,396198,SERVICIO DE PUBLICACION EN PRENSA ESCRITA Y ME...,16-07-21,168.0,"[154,174]",Municipalidad de Asunción,According
1,405074,REPARACION DE MODULO DE POTENCIA DE TABLERO DE...,17-03-22,394.0,"[68,88]",Empresa de Servicios Sanitarios del Paraguay (...,Overpriced
2,408866,Adquisición de Filtros Varios,17-11-22,541.0,"[328,348]",Industria Nacional del Cemento (INC),Overpriced
3,414679,DISEÑO Y READECUACION DE PLAYA Y COSTANERA DE ...,15-09-22,22000.0,"[105,125]",Municipalidad de Salto del Guairá,Overpriced
4,415044,LLAMADO MOPC N° 38/2022 LICITACION PUBLICA NAC...,09-08-22,500.0,"[231,251]",Ministerio de Obras Públicas y Comunicaciones ...,Overpriced
...,...,...,...,...,...,...,...
6395,428349,Servicio de Iluminacion y Sonido para escenari...,18-05-23,21.0,"[143,163]",Universidad Nacional de Asunción (UNA),Underprice
6396,428547,LPN N° 25/23 SERVICIOS TERCERIZADOS PARA EL MA...,08-05-23,3735.0,"[159,179]",Banco Central del Paraguay (BCP),Overpriced
6397,428577,LCO 60/23 - SERVICIO DE SEPELIO,27-04-23,320.0,"[187,207]",Banco Central del Paraguay (BCP),Overpriced
6398,429305,CONSTRUCCIÓN DE POLIDEPORTIVO MUNICIPAL EN EL ...,05-05-23,7565.0,"[152,172]",Gobierno Departamental de Alto Paraná (A.PARANÁ),Overpriced


In [ ]:
data1.to_excel(source+r"resultados_limpios.xlsx")

In [ ]:
inte={"trail":[]}